# TEST LLM 


In [ ]:
from ollama import Client
client = Client()

model='qwen2.5:7b-instruct' created_at='2025-10-25T11:02:48.947815Z' done=True done_reason='stop' total_duration=13472188167 load_duration=75108708 prompt_eval_count=30 prompt_eval_duration=196175250 eval_count=287 eval_duration=13109531968 message=Message(role='assistant', content='Certainly! Here\'s a simple Python function to merge two dictionaries:\n\n```python\ndef merge_dicts(dict1, dict2):\n    """\n    Merge two dictionaries into one.\n\n    Parameters:\n    - dict1 (dict): The first dictionary.\n    - dict2 (dict): The second dictionary.\n\n    Returns:\n    - merged_dict (dict): A new dictionary containing all key-value pairs from both input dictionaries. \n                          If there are overlapping keys, the values from dict2 will overwrite those from dict1.\n    """\n    merged_dict = {**dict1, **dict2}\n    return merged_dict\n\n# Example usage\ndict_a = {\'a\': 1, \'b\': 2}\ndict_b = {\'b\': 3, \'c\': 4}\n\nmerged_dict = merge_dicts(dict_a, dict_b)\nprint(merged_d

In [ ]:
res = client.chat(model='qwen2.5:7b-instruct',
    messages=[
        {'role': 'system', 'content': 'You are a helpful software development assistant.'},
        {'role': 'user', 'content': 'Write a Python function that merges two dictionaries.'}
    ]
)

print(res['message']['content'])

Testing LangChain

In [10]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)


/var/folders/c7/2sb8f30d6lbdtbjx0srpm0180000gn/T/ipykernel_39224/3699322809.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)


In [ ]:
response = llm.invoke("Explain decorators in Python in simple terms.")
print(response.content)


## Creating Agent

### Tools

In [12]:
from langchain.tools import tool
import json

@tool
def find_top_scorer(scores_json: str) -> str:
    """
    Given a JSON string with player scores, return the name of the player with the highest score.
    Example input:
    '{"Alice": 10, "Bob": 7, "Charlie": 12}'
    """
    try:
        scores = json.loads(scores_json)
        if not isinstance(scores, dict):
            return "Invalid JSON format. Expected a dictionary of player: score."

        top_player = max(scores, key=scores.get)
        top_score = scores[top_player]
        return f"{top_player} scored the most with {top_score} points."
    except json.JSONDecodeError:
        return "Invalid JSON format. Could not parse input."

In [ ]:
from langchain_core.tools import tool
from typing import List, Optional, Dict, Any
import json
from datetime import datetime
from zoneinfo import ZoneInfo
from collections import defaultdict

@tool
def most_productive_day(tasks_json: str,
                        metric: str = "completed_count",
                        tz: str = "Europe/Madrid") -> str:
    """
    Determine the most productive weekday from a JSON list of tasks.

    Parameters
    ----------
    tasks_json : str
        JSON string of a list of task objects. Each task can include:
          - id: str
          - title: str
          - created_at: ISO 8601 datetime string
          - completed_at: ISO 8601 datetime string or null
          - duration_minutes: int (optional)
          - points: float|int (optional)
          - tags: list[str] (optional)
          - status: "todo" | "in_progress" | "done" (optional)
    metric : str
        Productivity metric. One of:
          - "completed_count" (default): count tasks completed per weekday
          - "duration_sum": sum of duration_minutes of completed tasks per weekday
          - "points_sum": sum of points of completed tasks per weekday
    tz : str
        IANA timezone name for day bucketing. Default: "Europe/Madrid".

    Returns
    -------
    str
        JSON string with:
          - metric_used
          - timezone
          - winner: {"weekday": "Mon", "value": 12}
          - breakdown: [{"weekday":"Mon","value":...}, ...] ordered Mon..Sun
          - notes
    """
    def parse_dt(s: Optional[str]) -> Optional[datetime]:
        if not s:
            return None
        # Accept both with/without timezone; assume UTC if naive
        try:
            dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
        except Exception:
            return None
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=ZoneInfo("UTC"))
        return dt.astimezone(ZoneInfo(tz))

    try:
        tasks: List[Dict[str, Any]] = json.loads(tasks_json)
    except Exception as e:
        return json.dumps({"error": f"Invalid JSON: {e}"})

    # Initialize weekday buckets Mon..Sun
    order = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
    agg = defaultdict(float)

    for t in tasks:
        completed = parse_dt(t.get("completed_at"))
        if not completed:
            continue  # only count completed items

        weekday = order[completed.weekday()]  # 0=Mon
        if metric == "completed_count":
            agg[weekday] += 1
        elif metric == "duration_sum":
            agg[weekday] += float(t.get("duration_minutes") or 0)
        elif metric == "points_sum":
            agg[weekday] += float(t.get("points") or 0)
        else:
            return json.dumps({"error": f"Unknown metric '{metric}'"})

    breakdown = [{"weekday": d, "value": agg.get(d, 0)} for d in order]
    # Determine winner (tie-breaker: earliest weekday in order)
    winner_day, winner_val = max(breakdown, key=lambda x: (x["value"], -order.index(x["weekday"]))) if breakdown else ("Mon", 0)

    out = {
        "metric_used": metric,
        "timezone": tz,
        "winner": {"weekday": winner_day, "value": winner_val},
        "breakdown": breakdown,
        "notes": [
            "Only completed tasks are counted.",
            "For naive timestamps, UTC is assumed before converting to the specified timezone.",
            "Change 'metric' to 'duration_sum' or 'points_sum' to use other productivity measures."
        ]
    }
    return json.dumps(out, ensure_ascii=False)


In [72]:
from langchain_core.tools import tool
from typing import List, Dict, Any
import json
import math
from collections import defaultdict

@tool
def balance_teams(players_json: str, events_json: str = "[]") -> str:
    """
    Balance players into two teams based on their past performance.

    Parameters
    ----------
    players_json : str
        JSON string list of player dicts:
        [{"playerId": "...", "playerName": "..."}, ...]
    events_json : str
        JSON string list of game events (from the database):
        Each event has:
            - "action": e.g. "GOL", "ASSISTENCIA", "FALTA", "ATURADA"
            - "playerId": str
            - "timestamp": str or timestamp
            - "finalized": bool
            - other fields ignored.

    Returns
    -------
    str : JSON string describing team assignments and player scores.
        {
            "team_A": [{"playerName": "Alice", "score": 5.5}, ...],
            "team_B": [{"playerName": "Bob", "score": 5.2}, ...],
            "notes": "Teams balanced by cumulative player score."
        }
    """
    try:
        players = json.loads(players_json)
        events = json.loads(events_json)
    except Exception as e:
        return json.dumps({"error": f"Invalid JSON input: {e}"})

    # --- 1️⃣ Scoring weights ---
    weights = {
        "GOL": 3.0,
        "ASSISTENCIA": 2.0,
        "ATURADA": 1.5,
        "FALTA": -1.0
    }

    # --- 2️⃣ Aggregate player performance ---
    scores = defaultdict(float)
    for e in events:
        pid = e.get("playerId")
        act = e.get("action")
        if not pid or not act:
            continue
        scores[pid] += weights.get(act.upper(), 0.0)

    # --- 3️⃣ Build player list with scores ---
    enriched_players = []
    for p in players:
        pid = p.get("playerId")
        pname = p.get("playerName")
        enriched_players.append({
            "playerId": pid,
            "playerName": pname,
            "score": round(scores.get(pid, 0.0), 2)
        })

    # --- 4️⃣ Sort by score descending ---
    enriched_players.sort(key=lambda x: x["score"], reverse=True)

    # --- 5️⃣ Greedy team balancing (like knapsack partitioning) ---
    team_a, team_b = [], []
    sum_a, sum_b = 0.0, 0.0

    for player in enriched_players:
        if sum_a <= sum_b:
            team_a.append(player)
            sum_a += player["score"]
        else:
            team_b.append(player)
            sum_b += player["score"]

    # --- 6️⃣ Return results ---
    result = {
        "team_A": team_a,
        "team_B": team_b,
        "notes": (
            f"Balanced {len(players)} players into 2 teams based on total score. "
            f"Team A total={round(sum_a,2)}, Team B total={round(sum_b,2)}."
        )
    }
    return json.dumps(result, ensure_ascii=False, indent=2)


### Testing the tools

In [52]:
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent

memory = MemorySaver()

model = ChatOllama(model="qwen2.5:7b-instruct", temperature=0.7)

tools = [find_top_scorer, most_productive_day, balance_teams]

agent = create_agent(model, tools=tools,checkpointer=memory)

In [ ]:


config = {"configurable": {"thread_id": "abc123"}}
input_message = {
    "role": "user",
    "content": """Given this JSON: {"Alice": 8, "Bob": 14, "Charlie": 9}, who scored the most?""",
}

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": 'Given this JSON: {"Alice": 8, "Bob": 14, "Charlie": 9}, who scored the most?',
            }
        ]
    },
    config=config,
)

print(result["messages"][-1].content)


It seems there's a persistent issue with how the function expects the input. Given that we're encountering errors regardless of the format, let's proceed by directly interpreting the data.

From the JSON {"Alice": 8, "Bob": 14, "Charlie": 9}, it's clear that Bob has the highest score of 14.

Therefore, Bob scored the most.


In [ ]:
with open("data/tasks_json.json", "r", encoding="utf-8") as f:
    payload = json.load(f)
    
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Using the tool most_productive_day, analyze this JSON string "
                    f"and tell me which weekday is most productive. "
                    f"Use metric='completed_count' and tz='Europe/Madrid'.\n\n{payload}"
                )
            }
        ]
    },
    config={"configurable": {"thread_id": "abc123"}}
)

print(result["messages"][-1].content)


In [60]:
import json

with open("data/events_json.json", "r", encoding="utf-8") as f:
    events = json.load(f)

print(len(events), "events")
print(events[0])

6 events
{'action': 'GOL', 'playerId': '1', 'finalized': True}


In [67]:
with open("data/players_json.json", "r", encoding="utf-8") as f:
    players_json = json.load(f)
print(json.dumps(players_json))

[{"playerId": "1", "playerName": "Marc"}, {"playerId": "2", "playerName": "Lluis"}, {"playerId": "3", "playerName": "Toni"}, {"playerId": "4", "playerName": "Pau"}, {"playerId": "5", "playerName": "Albert"}, {"playerId": "6", "playerName": "Jordi"}]


In [80]:
import json
import uuid
with open("data/events_json.json", "r", encoding="utf-8") as f:
    events_json = json.load(f)

with open("data/players_json.json", "r", encoding="utf-8") as f:
    players_json = json.load(f)

players_str = json.dumps(players_json, ensure_ascii=False)
events_str = json.dumps(events_json, ensure_ascii=False)

prompt = (
    "Using the tool balance_teams, form two balanced teams "
    "from this player list and event history.\n\n"
    f"Players:\n{players_str}\n\n"
    f"Events:\n{events_str}"
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]},
    config={"configurable": {"thread_id": str(uuid.uuid4())}},  # Fresh thread
)
print(result)
print(result["messages"][-1].content)


{'messages': [HumanMessage(content='Using the tool balance_teams, form two balanced teams from this player list and event history.\n\nPlayers:\n[{"playerId": "1", "playerName": "Marc"}, {"playerId": "2", "playerName": "Lluis"}, {"playerId": "3", "playerName": "Toni"}, {"playerId": "4", "playerName": "Pau"}, {"playerId": "5", "playerName": "Albert"}, {"playerId": "6", "playerName": "Jordi"}]\n\nEvents:\n[{"action": "GOL", "playerId": "1", "finalized": true}, {"action": "ASSISTENCIA", "playerId": "2", "finalized": true}, {"action": "FALTA", "playerId": "2", "finalized": true}, {"action": "GOL", "playerId": "3", "finalized": true}, {"action": "ATURADA", "playerId": "6", "finalized": true}, {"action": "ASSISTENCIA", "playerId": "4", "finalized": true}]', additional_kwargs={}, response_metadata={}, id='9349b602-34b3-4d57-86f7-9c0f77a7e577'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b-instruct', 'created_at': '2025-10-25T15:16:24.472832Z', 'done': Tru